In [61]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.util import ngrams
from collections import Counter
import re
import numpy as np
import random
import math
import operator
import string

In [63]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# **DATA CLEANING AND PREPROCESSING**

In [0]:
file1 = open("/content/drive/My Drive/speeches.txt","r")
trainstr = file1.read()
trainstr = trainstr.lower()

In [65]:
tnew = sent_tokenize(trainstr)
for i in range (len(tnew)):
  tnew[i] = re.sub(r'[^A-Za-z\s\']+', "", tnew[i])
  tnew[i] = '<s> '+tnew[i]+' </s>'
train = tnew[:13000] # 80% of dataset is train
test = tnew[13000:] # 20% of dataset is test
print(len(tnew))

16401


# CLASSICAL APPROACH

In [0]:
fdist1 = {} #bigram
for i in range (len(train)):
  tokens = train[i].split()
  bigrams = nltk.bigrams(tokens)
  fdist = dict(nltk.FreqDist(bigrams))
  fdist1 = dict(Counter(fdist)+Counter(fdist1))

In [0]:
fdist2 = {} #trigram
for i in range (len(train)):
  tokens = train[i].split()
  trigrams = nltk.trigrams(tokens)
  fdist = dict(nltk.FreqDist(trigrams))
  fdist2 = dict(Counter(fdist)+Counter(fdist2))

In [0]:
from nltk import everygrams
fdist3 = {} #quadgram
for i in range (len(train)):
  tokens = train[i].split()
  fourgrams = list(everygrams(tokens,4,4)) 
  fdist = dict(nltk.FreqDist(fourgrams))
  fdist3 = dict(Counter(fdist)+Counter(fdist3))

In [173]:
trainstr = re.sub(r'[^A-Za-z\s\']+', "", trainstr)
ls = trainstr.split()
dictfinal = {}
for i in range (len(ls)):
  if ls[i] not in dictfinal:
    dictfinal[ls[i]] = 1
  else:
    dictfinal[ls[i]] += 1
Vocab = len(dictfinal)
Token = sum(dictfinal.values())
Vocab

6069

**MAXIMUM LIKELIHOOD ESTIMATIONS**

In [0]:
def MLEunigram(w1):
  if w1 not in d:
    return 1/(Vocab+Token) # Add 1 Smoothing
  else:
    return (d[w1]+1)/(Vocab+Token) # Add 1 Smoothing

In [0]:
def MLEbigram(w1,w2):
  if (w1+" "+w2) not in fdist1:
    if w1 in d.keys():
      return 1/(d[w1]+Vocab) # Add 1 Smoothing
    else:
      return 1/(Vocab) # Add 1 Smoothing
  else:
    if w1 in d.keys():
      return (fdist1(w1+" "+w2)+1)/(d[w1]+Vocab) # Add 1 Smoothing
    else:
      return (fdist1(w1+" "+w2)+1)/(Vocab) # Add 1 Smoothing

In [0]:
def MLEtrigram(w1,w2,w3):
  if (w1+" "+w2+" "+w3) not in fdist2:
    if w1+" "+w2 in fdist1.keys():
      return 1/(fdist1[w1+" "+w2]+Vocab) # Add 1 Smoothing
    else:
      return 1/(Vocab) # Add 1 Smoothing
  else:
    if w1+" "+w2 in fdist1.keys():
      return (fdist2(w1+" "+w2+" "+w3)+1)/(fdist1[w1+" "+w2]+Vocab) # Add 1 Smoothing
    else:
      return (fdist2(w1+" "+w2+" "+w3)+1)/(Vocab) # Add 1 Smoothing

In [0]:
def MLEquadgram(w1,w2,w3,w4):
  if (w1+" "+w2+" "+w3+" "+w4) not in fdist3:
    if w1+" "+w2 + " "+w3 in fdist2.keys():
      return 1/(fdist2[w1+" "+w2+" "+w3]+Vocab) # Add 1 Smoothing
    else:
      return 1/(Vocab) # Add 1 Smoothing
  else:
    if w1+" "+w2 + " "+w3 in fdlist2.keys():
      return (fdist3(w1+" "+w2+" "+w3+" "+w4)+1)/(fdist2[w1+" "+w2+" "+w3]+Vocab) # Add 1 Smoothing
    else:
      return (fdist3(w1+" "+w2+" "+w3+" "+w4)+1)/(Vocab) # Add 1 Smoothing

# CALCULATING SENTENCE PROBABILITIES USING UNIGRAM, BIGRAM, TRIGRAM AND QUADGRAM

In [0]:
def unigramsentenceprob(sentence):
  sentence_probability_log_sum = 0
  for word in sentence:
    x = MLEunigram(word)
    sentence_probability_log_sum += math.log(x,2)
  return math.pow(2, sentence_probability_log_sum)

In [0]:
def bigramsentenceprob(sentence):
  bigram_sentence_probability_log_sum = 0
  previous_word = None
  for word in sentence:
    if previous_word!=None:
      x = MLEbigram(previous_word,word)
      bigram_sentence_probability_log_sum += math.log(x,2)
    previous_word = word
  return math.pow(2, bigram_sentence_probability_log_sum)

In [0]:
def trigramsentenceprob(sentence):
  trigram_sentence_probability_log_sum = 0
  previous_word = None
  previous_previous_word = None
  for word in sentence:
    if previous_word!=None and previous_previous_word!=None:
      x = MLEtrigram(previous_previous_word,previous_word,word)
      trigram_sentence_probability_log_sum += math.log(x,2)
    previous_previous_word = previous_word
    previous_word = word
  return math.pow(2, trigram_sentence_probability_log_sum)

In [0]:
def quadgramsentenceprob(sentence):
  quadgram_sentence_probability_log_sum = 0
  previous_word = None
  previous_previous_word = None
  previous_previous_previous_word = None
  for word in sentence:
    if previous_word!=None and previous_previous_word!=None and previous_previous_previous_word!=None :
      x = MLEquadgram(previous_previous_previous_word,_previous_previous_word,previous_word,word)
      quadgram_sentence_probability_log_sum += math.log(x,2)
    previous_previous_previous_word = previous_previous_word
    previous_previous_word = previous_word
    previous_word = word
  return math.pow(2, quadgram_sentence_probability_log_sum)

In [0]:
def calculate_number_of_unigrams(sentences):
  unigram_count = 0
  for sentence in sentences:
    # remove two for <s> and </s>
    unigram_count += len(sentence) - 2
  return unigram_count
def calculate_number_of_bigrams(sentences):
  bigram_count = 0
  for sentence in sentences:
    bigram_count += len(sentence) - 1
  return bigram_count
def calculate_number_of_trigrams(sentences):
  trigram_count = 0
  for sentence in sentences:
    trigram_count += len(sentence) - 2
  return trigram_count
def calculate_number_of_quadgrams(sentences):
  quadgram_count = 0
  for sentence in sentences:
    quadgram_count += len(sentence) - 3
  return quadgram_count

# FUNCTIONS FOR CALCULATING PERPLEXITY FOR UNIGRAM, BIGRAM AND TRIGRAM MODELS

In [0]:
def calculate_unigram_perplexity(sentences):
  unigram_count = calculate_number_of_unigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(unigramsentenceprob(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / unigram_count)

In [0]:
def calculate_bigram_perplexity(sentences):
  bigram_count = calculate_number_of_bigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(bigramsentenceprob(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / bigram_count)

In [0]:
def calculate_trigram_perplexity(sentences):
  trigram_count = calculate_number_of_bigrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(trigramsentenceprob(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / trigram_count)

In [0]:
def calculate_quadgram_perplexity(sentences):
  quadgram_count = calculate_number_of_quadgrams(sentences)
  sentence_probability_log_sum = 0
  for sentence in sentences:
    try:
      sentence_probability_log_sum -= math.log(quadgramsentenceprob(sentence), 2)
    except:
      sentence_probability_log_sum -= 0
  return math.pow(2, sentence_probability_log_sum / quadgram_count)

# PERPLEXITY OF VARIOUS MODELS WITH CLASSICAL APPROACH

In [241]:
import math
print("Perplexity of test corpus with respect to unigram model is:",end = " ")
print(calculate_unigram_perplexity(test))
print("Perplexity of test corpus with respect to bigram model is:",end = " ")
print(calculate_bigram_perplexity(test))
print("Perplexity of test corpus with respect to trigram model is:",end = " ")
print(calculate_trigram_perplexity(test))
print("Perplexity of test corpus with respect to quadgram model is:",end = " ")
print(calculate_quadgram_perplexity(test))


Perplexity of test corpus with respect to unigram model is: 186.6357508882258
Perplexity of test corpus with respect to bigram model is: 147.40810983034604
Perplexity of test corpus with respect to trigram model is: 135.92047675682477
Perplexity of test corpus with respect to quadgram model is: 1.0


# CLASSICAL APPROACH FOR FUNCTION GENERATOR

In [0]:
def nc(sentences, ng):
  counterdict = dict()
  n = 0
  for i in range(len(sentences)):
    tokens = sentences[i].split(" ")
    for j in range(0,len(tokens)-ng+1):
      k = tuple(tokens[j: j+ng])
      if k not in counterdict:
        counterdict[k] = 1
        n += 1
      else:
        counterdict[k] += 1
  return [counterdict, n]

In [0]:
cu, nu= nc(tnew, 1) # unigram
cb, nb = nc(tnew, 2) # bigram
ct, nt = nc(tnew, 3) # trigram
cq, nq = nc(tnew, 4) # quadgram

def uni(word_list):
  prob = 0
  try:
    num = cu[tuple(word_list)]
    return num/nu
  except:
    return prob
def bi(word_list):
  prob = 0
  try:
    num = cb[tuple(word_list)]
    del word_list[-1]
    den = cu[tuple(word_list)]
    return num/den
  except:
    return prob
def tri(word_list):
  prob = 0
  try:
    num = ct[tuple(word_list)]
    del word_list[-1]
    den = cb[tuple(word_list)]
    return num/den
  except:
    return prob
def quad(word_list):
  prob = 0
  try:
    num = cq[tuple(word_list)]
    del word_list[-1]
    den = ct[tuple(word_list)]
    return num/den
  except:
    return prob

In [0]:
# Classical generator to generate new words using a n_gram model
def Generator(n_gram, initseq):
  sentence = []
  if(n_gram==1):
    for i in range(30):
      mp = 0
      mpl = list()
      for j in dictfinal.keys():
        k = (uni([j]))
        if(k>mp):
          mp = k
          mpl = [j]
        elif(k==mp):
          max_prob_list.append(j)
      samples = np.random.multinomial(30,[mp]*len(mpl),size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(mpl[index])
  else:
    sentence.extend(initseq)
    i = len(initseq)
    while(sentence[-1]!="</s>" and i<30):
      mp = 0
      mpl = list()
      for j in dictfinal.keys():
        word_list = sentence[(1-n_gram):]
        word_list.append(j)
        z1 = bi(word_list)
        z2 = tri(word_list)
        z3 = quad(word_list)
        if(n_gram==2):
          k = z1
        elif(n_gram==3):
          k = z2
        elif(n_gram==4):
          k = z3
        if(k>mp):
          mp = k
          mpl = [j]
        elif(k==mp):
          mpl.append(j)
      t = mp
      inp = [t]*len(mpl)
      samples = np.random.multinomial(30,inp,size=1)
      index, value = max(enumerate(samples), key = operator.itemgetter(1))
      sentence.append(mpl[index])
      i = i+1
    
  return ["<s>"]+sentence+["</s>"]

# RANDOM SENTENCES GENERATED USING UNIGRAM, BIGRAM, TRIGRAM AND QUADGRAM MODELS

In [248]:
print("Random Text generated by Unigram Model: ",end = "")
print(" ".join(Generator(1,["he"])))
print("Random Text generated by Bigram Model: ",end = "")
print(" ".join(Generator(2,["it", "should"])))
print("Random Text generated by Trigram Model: ",end = "")
print(" ".join(Generator(3,["there", "is", "always"])))
print("Random Text generated by Quadgram Model: ",end = "")
print(" ".join(Generator(4,["it", "should", "not", "be"])))

Random Text generated by Unigram Model: <s> the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the </s>
Random Text generated by Bigram Model: <s> it should have to be a lot of the way i dont know what i dont know what i dont know what i dont know what i dont know what </s>
Random Text generated by Trigram Model: <s> there is always again to bring our jobs back from china from mexico and i said i dont know what theyre doing it because they want to be a great </s>
Random Text generated by Quadgram Model: <s> it should not be allowed to run i would be very nice as far as im concerned i really dont know much about it thank thank thank thank thank thank </s>


In [273]:
print("The above 4 sentences generated by various approaches are not gramatically correct")

The above 4 sentences generated by various approaches are not gramatically correct


# NEURAL APPROACH

In [0]:
# Neural Approach

sequences = list()
n_gram = 3
for i in range(len(train)):
  tokens = train[i].split(" ")
  if(len(tokens)>=n_gram):
    for k in range(0,len(tokens)-n_gram+1):
      sequences.append(tokens[k:k+n_gram])

In [115]:
sequences[:10]

[['<s>', 'thank', 'you'],
 ['thank', 'you', 'so'],
 ['you', 'so', 'much'],
 ['so', 'much', '</s>'],
 ['<s>', "that's", 'so'],
 ["that's", 'so', 'nice'],
 ['so', 'nice', '</s>'],
 ['<s>', "isn't", 'he'],
 ["isn't", 'he', 'a'],
 ['he', 'a', 'great']]

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

In [0]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sequences)
integer_encoded_sequences = tokenizer.texts_to_sequences(sequences)

In [118]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5481


In [119]:
integer_encoded_sequences[:10]

[[3, 137, 10],
 [137, 10, 20],
 [10, 20, 96],
 [20, 96, 4],
 [3, 610, 20],
 [610, 20, 181],
 [20, 181, 4],
 [3, 2354, 35],
 [2354, 35, 7],
 [35, 7, 49]]

In [120]:
integer_encoded_sequences = np.array(integer_encoded_sequences)
np.shape(sequences)

(133651, 3)

In [0]:
X = integer_encoded_sequences[:,:-1]
y = integer_encoded_sequences[:,-1]

In [0]:
y = to_categorical(y, num_classes=vocab_size)

# VANILLA RNN MODEL

In [124]:
# Building Vanilla RNN Model
from keras.layers import SimpleRNN

rnn_model = Sequential()
rnn_model.add(Embedding(vocab_size, 50, input_length=n_gram-1))
rnn_model.add(SimpleRNN(35, return_sequences=True))
rnn_model.add(SimpleRNN(35))
rnn_model.add(Dense(60, activation='relu'))
rnn_model.add(Dense(vocab_size, activation='softmax'))

rnn_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 2, 50)             274050    
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 2, 35)             3010      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 35)                2485      
_________________________________________________________________
dense_9 (Dense)              (None, 60)                2160      
_________________________________________________________________
dense_10 (Dense)             (None, 5481)              334341    
Total params: 616,046
Trainable params: 616,046
Non-trainable params: 0
_________________________________________________________________


In [125]:
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
133651/133651 [==============================] - 21s 154us/step - loss: 5.6176 - acc: 0.1354
Epoch 2/20
133651/133651 [==============================] - 18s 137us/step - loss: 4.8511 - acc: 0.1907
Epoch 3/20
133651/133651 [==============================] - 20s 149us/step - loss: 4.5738 - acc: 0.2106
Epoch 4/20
133651/133651 [==============================] - 19s 144us/step - loss: 4.3761 - acc: 0.2241
Epoch 5/20
133651/133651 [==============================] - 18s 137us/step - loss: 4.2148 - acc: 0.2348
Epoch 6/20
133651/133651 [==============================] - 19s 145us/step - loss: 4.0806 - acc: 0.2445
Epoch 7/20
133651/133651 [==============================] - 19s 144us/step - loss: 3.9622 - acc: 0.2534
Epoch 8/20
133651/133651 [==============================] - 19s 138us/step - loss: 3.8565 - acc: 0.2615
Epoch 9/20
133651/133651 [==============================] - 20s 149us/step - loss: 3.7616 - acc: 0.2697
Epoch 10/20
133651/133651 [==============================] - 19s

# GENERATING SENTENCES ON TEST CORPUS USING RNN

In [267]:
generated_rnn = []
for i in range(len(test)):
  result = list()
  text = test[i].split(" ")
  f = 0
  if(len(text)>=n_gram):
    result = text[:n_gram-1]
    l = n_gram
    while(result[-1]!="</s>"):
      encoded = tokenizer.texts_to_sequences([result[-n_gram+1:]])
      encoded = np.array(encoded)
      yhat = rnn_model.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
      l+=1
    generated_rnn.append(" ".join(result))
generated_rnn[0:5]

['<s> so i dont know </s>',
 '<s> thats what i do </s>',
 '<s> thats what i do </s>',
 '<s> ive been a disaster </s>',
 '<s> all of the united states </s>']

In [271]:
print("The above 5 sentences are not gramatically and semantically correct")

The above 5 sentences are not gramatically and semantically correct


# LSTM MODEL

In [126]:
# Long Short Term Memory Model
lstm_model = Sequential()
lstm_model.add(Embedding(vocab_size, 50, input_length=n_gram-1))
lstm_model.add(LSTM(35, return_sequences=True))
lstm_model.add(LSTM(35))
lstm_model.add(Dense(55, activation='relu'))
lstm_model.add(Dense(vocab_size, activation='softmax'))

lstm_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2, 50)             274050    
_________________________________________________________________
lstm_5 (LSTM)                (None, 2, 35)             12040     
_________________________________________________________________
lstm_6 (LSTM)                (None, 35)                9940      
_________________________________________________________________
dense_11 (Dense)             (None, 55)                1980      
_________________________________________________________________
dense_12 (Dense)             (None, 5481)              306936    
Total params: 604,946
Trainable params: 604,946
Non-trainable params: 0
_________________________________________________________________


In [127]:
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X, y, batch_size=128, epochs=20)

Epoch 1/20
133651/133651 [==============================] - 32s 238us/step - loss: 6.1052 - acc: 0.0970
Epoch 2/20
133651/133651 [==============================] - 28s 208us/step - loss: 5.7382 - acc: 0.1042
Epoch 3/20
133651/133651 [==============================] - 30s 223us/step - loss: 5.3132 - acc: 0.1490
Epoch 4/20
133651/133651 [==============================] - 28s 213us/step - loss: 5.0113 - acc: 0.1750
Epoch 5/20
133651/133651 [==============================] - 27s 204us/step - loss: 4.8037 - acc: 0.1912
Epoch 6/20
133651/133651 [==============================] - 28s 206us/step - loss: 4.6463 - acc: 0.2023
Epoch 7/20
133651/133651 [==============================] - 27s 202us/step - loss: 4.5131 - acc: 0.2116
Epoch 8/20
133651/133651 [==============================] - 27s 204us/step - loss: 4.3924 - acc: 0.2193
Epoch 9/20
133651/133651 [==============================] - 29s 221us/step - loss: 4.2854 - acc: 0.2263
Epoch 10/20
133651/133651 [==============================] - 29s

# GENERATING SENTENCES ON TEST CORPUS USING LSTM

In [269]:
generated_lstm = []
for i in range(len(test)):
  result = list()
  text = test[i].split(" ")
  f = 0
  if(len(text)>=n_gram):
    result = text[:n_gram-1]
    l = n_gram
    while(result[-1]!="</s>"):
      encoded = tokenizer.texts_to_sequences([result[-n_gram+1:]])
      encoded = np.array(encoded)
      yhat = lstm_model.predict_classes(encoded, verbose=0)
      out_word = ''
      for word, index in tokenizer.word_index.items():
        if index == yhat:
          out_word = word
          break
      result.append(out_word)
      l+=1
    generated_lstm.append(" ".join(result))
generated_lstm[0:5]

['<s> so i said to the united states </s>',
 '<s> thats what we have to do it </s>',
 '<s> thats what we have to do it </s>',
 '<s> ive been a very very good </s>',
 '<s> all of the united states </s>']

In [272]:
print("The above 5 sentences are not gramatically and semantically correct")

The above 5 sentences are not gramatically and semantically correct


In [265]:
print("Perplexity of RNN on test corpus for trigram model is:",end = "")
print(print(calculate_trigram_perplexity(rnn_generated)))

print("Perplexity of LSTM on test corpus for trigram model is:",end = "")
print(print(calculate_trigram_perplexity(lstm_generated)))

Perplexity of RNN on test corpus for trigram model is:501.4042458789
Perplexity of LSTM on test corpus for trigram model is: 421.789546212


In [266]:
print("Perplexity of test corpus with respect to trigram model by classical approach is:",end = " ")
print(calculate_trigram_perplexity(test))

Perplexity of test corpus with respect to trigram model by classical approach is: 135.92047675682477


# Does Neural performs better than Classical, if so, why? If not, why not?

**Answer:** **As far as perplexity is concerned, LSTM performs better than RNN. ~H~owever perplexity values for neural approaches are higher than the classical approaches. This shows that the classical approaches are performing better. Grammatical sense is better in classical approaches. But neural approaches are able to generalize better bu semantic sense is worse. This can take in general n-1 words and predict the nth word which leads to less semantuc sense. However for sentences, it takes all previous words to produce the output and so more semantucally correct sentences are produced at the output.**